<a href="https://colab.research.google.com/github/jaydeepthik/PyTorch-implementations/blob/master/pytorch_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader

In [3]:
!nvidia-smi

Thu Dec 26 00:45:16 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
transformation = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.14,),(0.32,))])
train_data = datasets.MNIST('dataset/', train=True, transform=transformation, download=True)
test_data = datasets.MNIST('dataset/', train = False, transform=transformation, download=True)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(5*5*64, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)

        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)

        #x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        #x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


In [0]:
def fit_model(epoch, model, data_loader):
  for e in range(epoch):
    for batch_idx, (data, target) in enumerate(data_loader):
      data, target = data.to(device), target.to(device)
      #print(data.shape, target.shape)
      optimizer.zero_grad()
      output = model(data)
      #print(output.shape, target)
      loss = F.nll_loss(output, target)
      loss.backward()
      optimizer.step()

      if batch_idx % 1000 == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(e, batch_idx * len(data), len(train_loader.dataset),100. * batch_idx / len(train_loader), loss.item()))
    test(model, test_loader)

In [0]:
def test(model, data_loader):
  with torch.no_grad():
      model.eval()
      test_loss = 0
      correct =0

      for data, target in data_loader:
        data, target = data.to(device), target.to(device)
        output= model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).item()
        pred = output.max(1, keepdim=True)[1]
        correct+=pred.eq(target.view_as(pred)).sum().item()

      test_loss /= len(test_loader.dataset)
      print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset))) 


In [9]:
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters())

fit_model(10, model, train_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.307456
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.093539


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.0374, Accuracy: 9877/10000 (99%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.011178
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.095590

Test set: Average loss: 0.0377, Accuracy: 9878/10000 (99%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.003453
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.000665

Test set: Average loss: 0.0332, Accuracy: 9888/10000 (99%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.002122
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.154358

Test set: Average loss: 0.0271, Accuracy: 9914/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.006946
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.000335

Test set: Average loss: 0.0304, Accuracy: 9902/10000 (99%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.008311
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.049676

Test set: Average loss: 0.0461, Accuracy: 9868/10000 (99%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.004309
Train Epoch: 6 [32000/60000 (53%)]	Loss: 0.005535

Test set: Average loss: 0.0351, Accuracy: 9904/10000 (99%)

Tra

In [10]:
!nvidia-smi

Thu Dec 26 00:49:21 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    35W / 250W |    745MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  